# Setting Up Snowflake

In [ ]:
SET MY_USER = CURRENT_USER();

-- Check on this 
SET GITHUB_SECRET_USERNAME = '';
SET GITHUB_SECRET_PASSWORD = '';
SET GITHUB_URL_PREFIX = 'https://github.com/BigDataIA-Spring2025-4';
SET GITHUB_REPO_ORIGIN = 'https://github.com/BigDataIA-Spring2025-4/DAMG7245_Assignment03_Part02.git';

In [ ]:

USE ROLE ACCOUNTADMIN;

-- Roles
CREATE OR REPLACE ROLE FRED_ROLE;
GRANT ROLE FRED_ROLE TO ROLE SYSADMIN;
GRANT ROLE FRED_ROLE TO USER IDENTIFIER($MY_USER);


GRANT CREATE INTEGRATION ON ACCOUNT TO ROLE FRED_ROLE;
GRANT EXECUTE TASK ON ACCOUNT TO ROLE FRED_ROLE;
GRANT EXECUTE MANAGED TASK ON ACCOUNT TO ROLE FRED_ROLE;
GRANT MONITOR EXECUTION ON ACCOUNT TO ROLE FRED_ROLE;
GRANT IMPORTED PRIVILEGES ON DATABASE SNOWFLAKE TO ROLE FRED_ROLE;

-- Databases
CREATE OR REPLACE DATABASE FRED_DB;
GRANT OWNERSHIP ON DATABASE FRED_DB TO ROLE FRED_ROLE;

-- Warehouses
CREATE OR REPLACE WAREHOUSE FRED_WH WAREHOUSE_SIZE = XSMALL, AUTO_SUSPEND = 300, AUTO_RESUME= TRUE;
GRANT OWNERSHIP ON WAREHOUSE FRED_WH TO ROLE FRED_ROLE;

USE ROLE FRED_ROLE;
USE WAREHOUSE FRED_WH;
USE DATABASE FRED_DB;

In [ ]:
-- ----------------------------------------------------------------------------
-- Create the database level objects
-- ----------------------------------------------------------------------------


-- Schemas
CREATE OR REPLACE SCHEMA INTEGRATIONS;;
CREATE OR REPLACE SCHEMA DEV_RAW_FRED;
CREATE OR REPLACE SCHEMA DEV_HARMONIZED;
CREATE OR REPLACE SCHEMA DEV_ANALYTICS;
CREATE OR REPLACE SCHEMA PROD_RAW_FRED;
CREATE OR REPLACE SCHEMA PROD_HARMONIZED;
CREATE OR REPLACE SCHEMA PROD_ANALYTICS;

USE SCHEMA INTEGRATIONS;

CREATE OR REPLACE STAGE S3_FRED_STAGE
    URL = 's3://snowpipeline120/'
    CREDENTIALS = (AWS_KEY_ID = '' 
                   AWS_SECRET_KEY = '');


-- Secrets (schema level)
CREATE OR REPLACE SECRET FRED_GITHUB_SECRET
  TYPE = password
  USERNAME = $GITHUB_SECRET_USERNAME
  PASSWORD = $GITHUB_SECRET_PASSWORD;

-- API Integration (account level)
-- This depends on the schema level secret!
CREATE OR REPLACE API INTEGRATION FRED_GITHUB_API_INTEGRATION
  API_PROVIDER = GIT_HTTPS_API
  API_ALLOWED_PREFIXES = ($GITHUB_URL_PREFIX)
  ALLOWED_AUTHENTICATION_SECRETS = (FRED_GITHUB_SECRET)
  ENABLED = TRUE;

-- Git Repository
CREATE OR REPLACE GIT REPOSITORY FRED_GIT_REPO
  API_INTEGRATION = FRED_GITHUB_API_INTEGRATION
  GIT_CREDENTIALS = FRED_GITHUB_SECRET
  ORIGIN = $GITHUB_REPO_ORIGIN;

In [ ]:
USE ROLE ACCOUNTADMIN;

CREATE EVENT TABLE FRED_DB.INTEGRATIONS.FRED_EVENTS;
GRANT SELECT ON EVENT TABLE FRED_DB.INTEGRATIONS.FRED_EVENTS TO ROLE FRED_ROLE;
GRANT INSERT ON EVENT TABLE FRED_DB.INTEGRATIONS.FRED_EVENTS TO ROLE FRED_ROLE;

ALTER ACCOUNT SET EVENT_TABLE = FRED_DB.INTEGRATIONS.FRED_EVENTS;
ALTER DATABASE FRED_DB SET LOG_LEVEL = INFO;

In [ ]:
USE ROLE FRED_ROLE;
USE WAREHOUSE FRED_WH;
USE SCHEMA FRED_DB.INTEGRATIONS;

EXECUTE IMMEDIATE FROM @FRED_GIT_REPO/branches/main/scripts/deploy_notebooks.sql
    USING (env => 'DEV', schema1 => 'RAW_FRED', schema2 => 'HARMONIZED', schema3 => 'ANALYTICS', branch => 'main');

In [ ]:
EXECUTE NOTEBOOK FRED_DB.DEV_ANALYTICS."DEV_03_analytics_table_processing"()

In [ ]:
# To call the sproc
# from snowflake.snowpark.context import get_active_session
# session = get_active_session()

# session.use_schema("FRED_DB.DEV_ANALYTICS")
# # env = schema_name[:3]
# session.sql(f"CALL merge_fred_updates_sp('FRED_DB', 'DEV_ANALYTICS', 'DEV')").collect()

In [ ]:
# session.sql(f"CALL create_analytical_tables_sp('DEV_ANALYTICS', 'FRED_10Y_2Y')").collect()

In [ ]:
-- select * from FRED_10Y_2Y where OBSERVATION_DATE > '2025-02-24';

In [ ]:
-- USE ROLE ACCOUNTADMIN;
-- DROP ROLE FRED_ROLE;
-- DROP API INTEGRATION FRED_GITHUB_API_INTEGRATION;
-- DROP DATABASE FRED_DB;
-- DROP WAREHOUSE FRED_WH;

